In [ ]:
import os
import pandas as pd
import glob
cwd = os.getcwd()
import time

# Importing the BLM data (after downloading to your local pc)
df = pd.read_csv(r"C:\Users\mawal\OneDrive\Desktop\Twitter\Final_df\df_BLM.csv", encoding = "ISO-8859-1")

df.shape

In [ ]:
# Remove stopwords, etc then tokenize each tweet. 
#Then append each tokenized word to create a single list of all tweets (or unique words?)
import nltk
from nltk.tokenize import TweetTokenizer
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import sent_tokenize
import re
from nltk.corpus import stopwords
nltk.download('punkt')
import emoji

# Recoding the start time of the cell
start_run = time.time()
local_time = time.ctime(start_run)
print('start time: {}'.format(local_time))

# Converting the tweets from the text column into a single list
text_list = df['text'].tolist()

# Turning any punction into a period.
text_list2 = [re.sub(r'[,!?;-]+', '.', word) for word in text_list]

# Tokenizing the words into words in list
text_list3 = [nltk.word_tokenize(word) for word in text_list2]

# Flattening out the sublists to be a single list
text_list_sing = [i for j in text_list3 for i in j]

# Creating lists of lists
# Removing stopwords and non alpha characters from tokens
text_list4 = [[word.lower() for word in sentence if word.isalpha()
                or word == '.' or emoji.get_emoji_regexp().search(word)
                or word not in stopwords.words('english')] for sentence in text_list3]

# Removing stopwords and non alpha characters from tokens in single list
text_list_sing = [word.lower() for word in text_list_sing if word.isalpha()
                or word == '.' or emoji.get_emoji_regexp().search(word)
                or word not in stopwords.words('english')]


# Lemmatizing the tokens
lemma = WordNetLemmatizer()
text_list5 = [[lemma.lemmatize(word) for word in sentence] for sentence in text_list4]
text_list_sing = [lemma.lemmatize(word) for word in text_list_sing]


# Creating an indexed vocab list
vocab_indexed = {'__PAD__':0, '__</e>__': 1, '__UNK__':2}
for word in text_list_sing:
    if word not in vocab_indexed:
        vocab_indexed[word] = len(vocab_indexed)
    

# Creating the frequency distribution of words
fdist = nltk.FreqDist(word for word in text_list_sing)

# Creating the vocabulary of unique words
vocab = list(set(text_list_sing))

# Calculaing the end time and total run time
end_run = time.time()
local_time = time.ctime(end_run)
print('end time: {}'.format(local_time))
duration_run = round((end_run - start_run)/60, 2)
print('Total run time: {}'.format(duration_run))

print('size of the vocabulary: {}'.format(len(vocab)))

fdist.most_common(20)

In [ ]:
# Using text_list5 to create embeddings
sent = text_list5

# Using gensims phrases to create bigrams from text_list5
from gensim.models.phrases import Phrases, Phraser
phrases = Phrases(sent, min_count = 5, progress_per = 1000)
bigram = Phraser(phrases)
sentences = bigram[sent]

from gensim.models import Word2Vec
import multiprocessing

scores = multiprocessing.cpu_count()
model = Word2Vec(min_count = 5, window = 5, size = 300, sample = 6e-5,
                   alpha = 0.03, min_alpha = 0.0007, negative = 20, workers = scores-1)

# https://www.kaggle.com/pierremegret/gensim-word2vec-tutorial
# The parameters:
# min_count = int - Ignores all words with total absolute frequency lower than this - (2, 100)
# window = int - The maximum distance between the current and predicted word within a sentence. E.g. window words on the left and window words on the left of our target - (2, 10)
# size = int - Dimensionality of the feature vectors. - (50, 300)
# sample = float - The threshold for configuring which higher-frequency words are randomly downsampled. Highly influencial. - (0, 1e-5)
# alpha = float - The initial learning rate - (0.01, 0.05)
# min_alpha = float - Learning rate will linearly drop to min_alpha as training progresses. To set it: alpha - (min_alpha * epochs) ~ 0.00
# negative = int - If > 0, negative sampling will be used, the int for negative specifies how many "noise words" should be drown. If set to 0, no negative sampling is used. - (5, 20)
# workers = int - Use these many worker threads to train the model (=faster training with multicore machines)

model.build_vocab(sentences, progress_per = 1000)

model.train(sentences, total_examples = model.corpus_count, epochs = 30, report_delay = 1)

print(model.wv.vocab)

model.init_sims(replace = True)

print(model.wv.most_similar(positive = ['black']))

In [ ]:
from gensim.models import KeyedVectors

model.wv.save_word2vec_format('model.bin', binary = True)
reloaded_vectors = KeyedVectors.load_word2vec_format('model.bin', binary = True)

In [ ]:
reloaded_vectors